# Gather Data from Various Sources

## ChatGPT

Yes, I know. Model degeneration. Wtvr just let me test my model. ok thx bye.

In [3]:
from bs4 import BeautifulSoup
import pandas as pd

file_path = './raw/chat.html'  # Replace with the path to your HTML file
with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()

soup = BeautifulSoup(content, 'html.parser')

a_name = "flutter"
b_name = "react-native"

a_language = "dart"
b_language = "javascript"

a_code = []
b_code = []

# Find all <pre> elements with class 
messages = soup.find_all('pre')

for message in messages:
    text = message.text
    if '```' not in text: continue
    # Split the text into code blocks
    code_blocks = text.split('```')
    # get rid of 2 spaces at the beginning of each line
    code_blocks = [code.replace('\n  ', '\n') for code in code_blocks]
    # split by the first newline character
    a_code.append(code_blocks[1][len(f'{a_language}\n'):])
    b_code.append(code_blocks[3][len(f'{b_language}\n'):])

df = pd.DataFrame({a_name: a_code, b_name: b_code})
df.to_csv(f'raw/samples_{a_name}_{b_name}.csv', index=False)
df.tail()

flutter  \
30    import 'package:flutter/material.dart';\n  i...   
31    import 'package:flutter/material.dart';\n  i...   
32    import 'package:flutter/material.dart';\n  i...   
33    import 'package:flutter/material.dart';\n  i...   
34    import 'package:flutter/material.dart';\n  i...   

                                         react-native  
30    import React, { useState, useEffect } from '...  
31    import React, { useState, useEffect } from '...  
32    import React, { useState, useEffect } from '...  
33    import React, { createContext, useContext, u...  
34    import React, { useState, useEffect } from '...

### Load Data

In [2]:
df = pd.read_csv('raw/samples.csv')
print(df['dart'][0])

  List<int> twoSum(List<int> nums, int target) {
    Map<int, int> map = {};
    for (int i = 0; i < nums.length; i++) {
      int complement = target - nums[i];
      if (map.containsKey(complement)) {
        return [map[complement]!, i];
      }
      map[nums[i]] = i;
    }
    throw ArgumentError("No two sum solution");
  }
  


## Rosetta Code

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

# URL of the Rosetta Code category page for Dart
url = 'https://rosettacode.org/wiki/Category:Dart'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Extract task links
tasks = soup.find_all('a', href=True, title=True)
task_links = ['https://rosettacode.org' + task['href'] for task in tasks if 'Category' not in task['href'] and str(task['href']).startswith('/wiki/')][5:]

data = []

for link in task_links[:10]:
    print(link)
    task_response = requests.get(link)
    task_soup = BeautifulSoup(task_response.content, 'html.parser')

    # Find Dart and JavaScript code sections
    dart_code = None
    js_code = None

    code_sections = task_soup.find_all('pre')
    for code_section in code_sections:
        code = code_section.text
        print(code)
        print("##################")
        # Check if the immediate previous sibling h2 element contains the text "Dart"
        if code_section.find_previous_sibling('h2') and 'Dart' in code_section.find_previous_sibling('h2').text:
            dart_code = code
            print("Dart\n")
            # print(dart_code)
        # Check if the immediate previous sibling h2 element contains the text "JavaScript"
        if code_section.find_previous_sibling('h2') and 'JavaScript' in code_section.find_previous_sibling('h2').text:
            js_code = code
            print("JavaScript\n")
            # print(js_code)
    break

    if dart_code and js_code:
        data.append({'Task': link.split('/')[-1], 'Dart': dart_code, 'JavaScript': js_code})

# Create DataFrame
df_samples = pd.DataFrame(data)

# Display the DataFrame
print(df_samples)


https://rosettacode.org/wiki/100_doors
V doors = [0B] * 100
L(i) 100
   L(j) (i .< 100).step(i + 1)
      doors[j] = !doors[j]
   print(‘Door ’(i + 1)‘: ’(I doors[i] {‘open’} E ‘close’))
##################
*        100 doors                 13/08/2015
HUNDOOR  CSECT
         USING  HUNDOOR,R12
         LR     R12,R15
         LA     R6,0
         LA     R8,1               step 1
         LA     R9,100
LOOPI    BXH    R6,R8,ELOOPI       do ipass=1 to 100 (R6)
         LR     R7,R6
         SR     R7,R6
         LR     R10,R6             step ipass
         LA     R11,100
LOOPJ    BXH    R7,R10,ELOOPJ      do idoor=ipass to 100 by ipass (R7)
         LA     R5,DOORS-1
         AR     R5,R7
         XI     0(R5),X'01'        doors(idoor)=not(doors(idoor))
NEXTJ    B      LOOPJ
ELOOPJ   B      LOOPI
ELOOPI   LA     R10,BUFFER         R10 address of the buffer
         LA     R5,DOORS           R5 address of doors item
         LA     R6,1               idoor=1 (R6)
         LA     R9,100  

## LeetCode

The plan here is to scrape leetcode problems and their solutions. This will be done using the leetcode API.

Steps:

1. Sign in to leetcode.
2. Go to the [leetcode API page](https://leetcode.com/api/problems/all/).
3. Copy the `csrftoken` and `LEETCODE_SESSION` cookies.
4. Use the following code to get the problems and their solutions.

In [11]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import requests
import os

def get_from_env_file(key):
    with open('.env', 'r') as file:
        for line in file:
            if key in line:
                return line.split('=')[1].strip()

# first, we need to get the list of all leetcode problems
url = 'https://leetcode.com/api/problems/all/'
response = requests.get(url)
# wait for 5 seconds
time.sleep(5)
# if the request was not successful, save the response to a file
if response.status_code != 200:
    with open('x.html', 'w') as file:
        file.write(response.text)
    raise Exception(f'Failed to fetch LeetCode problems: {response.text}')
problems = response.json()['stat_status_pairs']
problems = pd.DataFrame(problems)
problems['title'] = problems['stat'].apply(lambda x: x['question__title'])
problems['title_slug'] = problems['stat'].apply(lambda x: x['question__title_slug'])
problems['difficulty'] = problems['difficulty'].apply(lambda x: x['level'])
problems = problems[['title', 'title_slug', 'difficulty']]
problems.head()

Exception: Failed to fetch LeetCode problems: <!DOCTYPE html><html lang="en-US"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131}button,html{font-family:system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji}@media (prefers-color-scheme:dark){body{background-color:#222;color:#d9d9d9}body a{color:#fff}body a:hover{color:#ee730a;text-decoration:underline}body .lds-ring div{border-color:#999 transparent transparent}body .font-red{color:#b20f03}body .pow-button{background-color:#4693ff;color:#1d1d1d}body #challenge-success-text{background-image:url